In [1]:
import pysam
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt

In [2]:
vcf = pysam.VariantFile('/project/jnovembre/jhmarcus/ancient-sardinia/output/vcf/ancient_sardinia_full26_trm.vcf.gz')

In [3]:
samples = list(vcf.header.samples)
cnt = 0
for rec in vcf.fetch():
    cnt += 1
nsnps = cnt

In [16]:
l = (1/3) * np.ones(shape = (len(samples), nsnps, 3))

In [5]:
# j index SNP
shape = (len(samples), nsnps)
mask = np.ones(shape)
j = 0
eps = 0.001
for rec in vcf.fetch():
    for s in range(len(samples)):
        ref = rec.samples[samples[s]]["AD"][0]
        alt = rec.samples[samples[s]]["AD"][1]
        for g in [0,1,2]:
            p        = (1-eps)*(g/2)   + eps*(1-g/2)
            l[s,j,g] = alt*np.log10(p) + ref*np.log10(1-p)
    j += 1

In [10]:
def compute_gradient(f, refs, alts, eps = 0.01):
    dl = 0
    for i in range(len(refs)):
        num   = 2*f * np.power(1-eps, alts[i]) * np.power(eps,refs[i]) + \
        2*(1-2*f)*np.power(0.5, alts[i])*np.power(0.5, refs[i]) + \
        (2*f-2) * np.power(eps, alts[i]) * np.power(1-eps, refs[i])
        
        denom = np.power(f, 2) * np.power(1-eps, alts[i]) * np.power(eps, refs[i]) + \
        2*f*(1-f)*np.power(0.5, alts[i])*np.power(0.5, refs[i]) + \
        np.power(1-f, 2) * np.power(eps, alts[i]) * np.power(1-eps, refs[i])
        dl = dl + (num/denom)
    return(dl)

def compute_loglikelihood(f, refs, alts, eps = 0.01):
    ll = 0
    for i in range(len(refs)):
        ll += np.log( np.power(f, 2)*np.power(1-eps, alts[i])*np.power(eps, refs[i]) + \
                    2*f*(1-f)*np.power(0.5, alts[i])*np.power(0.5, refs[i]) + \
                    np.power(1-f, 2) * np.power(eps, alts[i]) * np.power(1-eps, refs[i]))
    return(ll)

In [11]:
# testing
c   = np.random.poisson(1, 20)
alt = np.random.binomial(n = c, p = 0.5, size = 20)
ref = c - alt
root = optimize.brentq(compute_gradient, a=0, b=1, args = (ref,alt), xtol = 0.025)
print(root)

0.5409429060313207


In [6]:
frequencies = np.ones(nsnps)*0.5

In [13]:
i = 0
for rec in vcf.fetch():
    refs = np.zeros(len(samples))
    alts = np.zeros(len(samples))
    for s in range(len(samples)):
        refs[s] = rec.samples[samples[s]]["AD"][0]
        alts[s] = rec.samples[samples[s]]["AD"][1]
    phat = optimize.brentq(compute_gradient, a=0, b=1, args = (refs,alts), xtol = 0.025)
    frequencies[i] = phat
    i += 1

ValueError: f(a) and f(b) must have different signs

In [9]:
PP = np.zeros(shape = (len(samples), nsnps, 3))
for i in range(len(samples)):
    for j in range(nsnps):
        PP[i,j,0] = l[i,j,0] * np.power(1-frequencies[j], 2)
        PP[i,j,1] = l[i,j,1] * 2 * frequencies[j] * (1 - frequencies[j])
        PP[i,j,2] = l[i,j,2] * np.power(frequencies[j], 2)
        PP[i,j,:] = PP[i,j,:] / PP[i,j,:].sum()

/home/halasadi/anaconda3/envs/var/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [12]:
print(l[1,1,:])

[-4.34511774e-04 -3.01029996e-01 -3.00000000e+00]


In [13]:
print(PP[1,1,:])

[1.20614139e-04 1.67123084e-01 8.32756302e-01]


In [15]:
%load_ext Cython

In [ ]:
%%cython

cimport cython
import numpy as np
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double compute_distance(int i, int j, double[:, :, :] P, double [:,:] mask):
    cdef double d = 0.0
    cdef int nsnps = P.shape[1]
    cdef int l  = 0
    cdef int k1 = 0
    cdef int k2 = 0
    for l in range(nsnps):
        
        if (mask[i,l]):
            continue
            
        for k1 in [0,1,2]:
            for k2 in [0,1,2]:
                d += (k1-k2)*(k1-k2) * P[i, l, k1] * P[j, l, k2]
                
    return(d)

In [ ]:
shape = (len(samples), nsnps)
mask = np.ones(shape)

D = np.zeros(shape = (len(samples), len(samples)))
for i in range(len(samples)):
    for j in range((i+1), len(samples)):
            D[i,j] = compute_distance(i,j, P, mask)
            D[j,i] = D[i,j]
D = D / nsnps